In [249]:
import mwapi
import datetime as dt
from requests_oauthlib import OAuth1
import secrets.oauth as oauth_cfg
import re
import time

# Configuration

For multilingual messages, `MESSAGE` should be the output from "Assemble multilingual message" Lua model, 
as described at [meta:Newsletters/Translation#Regular process](https://meta.wikimedia.org/wiki/Newsletters/Translation#Regular_process).

For example:
```
{{#invoke:
Assemble multilingual message|
assembleMessage|
marker=ce-insights-content|
page=Community Engagement Insights/MassMessages/First message|
ar|de|es|fr|it|ja|nl|pl|pt|ru|uk|zh
}}
```

In [302]:
# A descriptive user agent for our API requests
USER_AGENT = (
    "CE Insights survey bot -- " +
    "https://github.com/wikimedia-research/Community-Engagement-Insights-sampling"
)

# A MassMessage list for basic testing. 
TEST_LIST = "User:Neil P. Quinn-WMF/MassMessage test list"

# The message text you want to send
MESSAGE = open("data/raw/first-message.txt").read()

# Number of seconds to wait before sending each individual MassMessage
WAITING_PERIOD = 15

# Load strata

In [304]:
real_strata = pd.read_csv("data/interim/sampled-strata.tsv", sep = "\t")
real_strata.head()

,proj_group,edit_bin,sample_size,sampled_users,page_title,survey_url,preview_url
0,arwiki,"[10, 30)",26,"[user(user='Michael M. Markos', proj_domain='a...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
1,arwiki,"[30, 150)",94,"[user(user='المسرحي', proj_domain='ar.wikipedi...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
2,arwiki,"[150, 600)",96,"[user(user='Budur Subaie', proj_domain='ar.wik...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
3,arwiki,"[600, 1200)",56,"[user(user='Ahmed El-Farash', proj_domain='ar....",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
4,arwiki,"[1200, 3500)",68,"[user(user='Ibrahim Old', proj_domain='ar.wiki...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...


In [240]:
test_strata = pd.read_csv("data/interim/test-strata.tsv", sep = "\t")
test_strata.head()

,proj_group,edit_bin,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)","[[""Neil P. Quinn-WMF"", ""ar.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ar1
1,asia_wps,"[30, 150)","[[""Neil P. Quinn-WMF"", ""ko.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/as2
2,cee_wps,"[150, 600)","[[""Neil P. Quinn-WMF"", ""uk.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ce3
3,commons,"[600, 1200)","[[""Neil P. Quinn-WMF"", ""commons.wikimedia.org""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/co4
4,dewiki,"[1200, 3500)","[[""Neil P. Quinn-WMF"", ""de.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/de5


# Set up connection

Use OAuth to authenticate as an [owner-only consumer](https://www.mediawiki.org/wiki/OAuth/Owner-only_consumers).

In [254]:
sess = mwapi.Session("https://meta.wikimedia.org", user_agent = USER_AGENT)

auth = OAuth1(
    oauth_cfg.consumer_token,
    oauth_cfg.consumer_secret,
    oauth_cfg.access_token,
    oauth_cfg.access_secret
)

def get_token():
    resp = sess.get(
        action="query", 
        meta="tokens", 
        type="csrf", 
        auth = auth
    )
    
    return resp["query"]["tokens"]["csrftoken"]

def api_get(*args, **kwargs):
    return sess.get(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        **kwargs
    )
    
def api_post(*args, **kwargs):
    return sess.post(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        token = get_token(),
        **kwargs
    )

# Check API authentication 

In [256]:
api_get(
    action = "query",
    meta = "userinfo"
)

{'batchcomplete': True,
 'query': {'userinfo': {'id': 19115528, 'name': 'WMF Surveys'}}}

# Assemble message

In [298]:
# Replace links
def add_link(msg, link):
    return re.sub(
        r"https://www.example.com",
        link,
        msg
    )

# Set up messaging

In [259]:
def send_message(target_list, subj, msg):
    api_post(
        action = "massmessage",
        spamlist = target_list,
        subject = subj,
        message = msg
    )


def message_frame(frame):
    for row in frame.itertuples():
        message_list = row.page_title
        survey_url = row.survey_url
        preview_url = row.preview_url
        
        linked_msg = add_link(msg, survey_url)
        
        print_err(
            "Sending to {message_list} with url {survey_url} in {wait} seconds".format(
                message_list = message_list,
                survey_url = survey_url,
                wait = WAITING_PERIOD
            )
        )
    
        time.sleep(WAITING_PERIOD)
        
        send_message(message_list, "Placeholder for multilingual subject", linked_msg)
        
        print_err("Sent.")

# Test messaging

## Basic test

In [299]:
send_message(
    TEST_LIST,
    "Testing MassMessage",
    manual_msg
)

## Test with full message

In [260]:
message_frame(test_strata)

Sending to User:Neil P. Quinn/2018 CE Insights/ar1 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ar&edc=1&prjedc=ar1 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/as2 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=as&edc=2&prjedc=as2 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/ce3 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ce&edc=3&prjedc=ce3 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/co4 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=co&edc=4&prjedc=co4 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/de5 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=de&edc=5&prjedc=de5 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/en6 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=en&edc=6&pr

KeyboardInterrupt: 

# ACTUAL SENDING

In [65]:
def massMessageAll():
    print_err("messaging 1")
    print_err("messaging 2")

In [66]:
massMessageAll()

messaging 1
messaging 2


In [275]:
# Run this cell and follow the prompt to unlock sending code for 30 seconds
ipyd.HTML("""
<script type="text/Javascript">
var kernel = IPython.notebook.kernel;

if (prompt("The code below will send talk pages messages to thousands of editors. Type 'run' to unlock it")) {
    kernel.execute("allow_run = dt.datetime.now()")
};
</script
""")

In [276]:
# If the cell above has been run within the past 30 seconds, send the messages
if (dt.datetime.now() - allow_run) < datetime.timedelta(seconds = 30):
    print_err("Sending!")
else:
    print_err(
        "This code will send talk page messages to thousands of editors.", 
        "If you're certain you want to do this, run the cell above to unlock it.",
        sep = "\n"
    )

Sending!
